In [156]:
import pandas as pd
import geopandas as gpd
import json
import numpy as np
import re

In [157]:
df = pd.read_csv("../data/raw_data.csv")
display(df.head(3))
display(df.columns)

,StartDate,EndDate,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,...,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21
0,Start Date,End Date,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,Recipient First Name,Recipient Email,...,Who did you vote for President,Were you excited to vote for your presidential...,How confident are you that your Rhode Island p...,How confident are you in the ability of local ...,How confident are you that the results of the ...,Where do you primarily get your information ab...,Why did you choose to vote in-person on electi...,Would you support Rhode Island implementing sa...,Would you support Rhode Island implementing ra...,NaN
1,11/12/2024 18:19:16,11/12/2024 18:21:01,100,104,True,11/12/2024 18:21:02,R_7LvyIKEDwhcpXXd,NaN,NaN,NaN,...,Kamala Harris,Somewhat Excited,Somewhat Confident,Somewhat Confident,Very Confident,NaN,Today was the best option for my schedule,Definitely,I would need more information to make a decision,NaN
2,11/12/2024 18:19:22,11/12/2024 18:21:06,100,103,True,11/12/2024 18:21:06,R_3obsi8B2j0maSsH,NaN,NaN,NaN,...,Kamala Harris,Extremely Excited,Not at All Confident,Somewhat Confident,NaN,NaN,NaN,Definitely,Definitely,NaN


Index(['StartDate', 'EndDate', 'Progress', 'Duration (in seconds)', 'Finished',
       'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName',
       'RecipientEmail', 'ExternalReference', 'LocationLatitude',
       'LocationLongitude', 'DistributionChannel', 'UserLanguage', 'Q1', 'Q2',
       'Q3', 'Q4', 'Q5', 'Q6', 'Unnamed: 21', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21'],
      dtype='object')

In [158]:
df.iloc[:,14:].head()

,UserLanguage,Q1,Q2,Q3,Q4,Q5,Q6,Unnamed: 21,Q7,Q8,...,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21
0,User Language,Which ward is this survey from?,What is your age group?,What is your gender identity?,What is your marital status?,With what ethnicity or race(s) do you most ide...,What is your religious affiliation? - Selected...,What is your religious affiliation? - Other - ...,What is your highest level of education?,What is your annual household income?,...,Who did you vote for President,Were you excited to vote for your presidential...,How confident are you that your Rhode Island p...,How confident are you in the ability of local ...,How confident are you that the results of the ...,Where do you primarily get your information ab...,Why did you choose to vote in-person on electi...,Would you support Rhode Island implementing sa...,Would you support Rhode Island implementing ra...,NaN
1,EN,5,30-39,Female,Not married,"Black or African American,American Indian or A...",Protestant / Other Christian Denomination,NaN,4 year degree,"$30,000-$49,999",...,Kamala Harris,Somewhat Excited,Somewhat Confident,Somewhat Confident,Very Confident,NaN,Today was the best option for my schedule,Definitely,I would need more information to make a decision,NaN
2,EN,15,30-39,Male,Not married,NaN,NaN,NaN,4 year degree,"$30,000-$49,999",...,Kamala Harris,Extremely Excited,Not at All Confident,Somewhat Confident,NaN,NaN,NaN,Definitely,Definitely,NaN
3,EN,9,40-49,Male,Not married,Asian,NaN,NaN,Professional degree,"$100,000-$199,999",...,Other,Not Very Excited,Somewhat Confident,Somewhat Confident,Not Very Confident,"Rhode Island Secretary of State's Website,Soci...",I wanted to wait to learn more about the candi...,Definitely,Definitely,NaN
4,EN,1,18-24,Male,Not married,Asian,NaN,NaN,Some college,"$100,000-$199,999",...,Kamala Harris,Not Very Excited,Not Very Confident,Very Confident,Somewhat Confident,Friends/Family,"I prefer voting on election day,I forgot to vo...",Definitely,Probably Not,NaN


The first row contains the actual question text. Let's create a mapping between the column and the question text, and then drop the first row. We may want to reference it later.

In [159]:
question_columns = list(df.iloc[:,14:-1].columns)
question_text = df.iloc[0,14:-1].values
question_mapping = dict(zip(question_columns, question_text))
question_mapping

{'UserLanguage': 'User Language',
 'Q1': 'Which ward is this survey from?',
 'Q2': 'What is your age group?',
 'Q3': 'What is your gender identity?',
 'Q4': 'What is your marital status?',
 'Q5': 'With what ethnicity or race(s) do you most identify? (Check all that apply)',
 'Q6': 'What is your religious affiliation? - Selected Choice',
 'Unnamed: 21': 'What is your religious affiliation? - Other - Text',
 'Q7': 'What is your highest level of education?',
 'Q8': 'What is your annual household income?',
 'Q9': 'How would you describe your political affiliation?',
 'Q10': 'Overall, do you feel that the country is heading in the right direction or the wrong direction?',
 'Q11': 'Overall, do you feel that Rhode Island is heading in the right direction or the wrong direction?',
 'Q12': 'Who did you vote for President',
 'Q13': 'Were you excited to vote for your presidential candidate?',
 'Q14': 'How confident are you that your Rhode Island politicians care about you?',
 'Q15': 'How confiden

In [160]:
with open('../data/question_mapping.json', 'w') as f:
    json.dump(question_mapping, f)

Data Cleaning and Preprocessing. We need to get the data where it looks like:

ward | feature_1_value_1 | feature_1_value_2 | ... | feature_2_value_1 | ...

1    | 35%               | 45%               | ... | 70%               | ...

So we first have to encode all the columns, and then aggregate them by ward to reach the desired state before we can it with our GIS data.

But does this mean we have to categorically encode all the features, even the ordinal ones?

In [161]:
# Drop columns and first row
df = df.iloc[:,14:-1]
df = df.drop(0).reset_index()
df = df.rename(columns={"Unnamed: 21":"Q6_other"})

In [162]:
display(df.iloc[:,:10].head())
display(df.iloc[:,10:].head())

,index,UserLanguage,Q1,Q2,Q3,Q4,Q5,Q6,Q6_other,Q7
0,1,EN,5,30-39,Female,Not married,"Black or African American,American Indian or A...",Protestant / Other Christian Denomination,NaN,4 year degree
1,2,EN,15,30-39,Male,Not married,NaN,NaN,NaN,4 year degree
2,3,EN,9,40-49,Male,Not married,Asian,NaN,NaN,Professional degree
3,4,EN,1,18-24,Male,Not married,Asian,NaN,NaN,Some college
4,5,EN,3,18-24,Female,Not married,Hispanic or Latino/a,NaN,NaN,Some college


,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
0,"$30,000-$49,999",Democrat,Unsure,Somewhat Right,Kamala Harris,Somewhat Excited,Somewhat Confident,Somewhat Confident,Very Confident,NaN,Today was the best option for my schedule,Definitely,I would need more information to make a decision
1,"$30,000-$49,999",Democrat,Definitely Right,Somewhat Wrong,Kamala Harris,Extremely Excited,Not at All Confident,Somewhat Confident,NaN,NaN,NaN,Definitely,Definitely
2,"$100,000-$199,999",Independent,Definitely Wrong,Somewhat Right,Other,Not Very Excited,Somewhat Confident,Somewhat Confident,Not Very Confident,"Rhode Island Secretary of State's Website,Soci...",I wanted to wait to learn more about the candi...,Definitely,Definitely
3,"$100,000-$199,999",Independent,Somewhat Right,Somewhat Right,Kamala Harris,Not Very Excited,Not Very Confident,Very Confident,Somewhat Confident,Friends/Family,"I prefer voting on election day,I forgot to vo...",Definitely,Probably Not
4,"Less than $30,000",Democrat,Definitely Wrong,Somewhat Right,Kamala Harris,Somewhat Excited,Somewhat Confident,Very Confident,Somewhat Confident,"Rhode Island Secretary of State's Website,Soci...","I prefer voting on election day,I forgot to vo...",Definitely,Probably


Let's zero in on the Q6 and Q6 other questions because mapping those might be a bit weird. They correspond to the religious affiliation questions.

In [163]:
#df[df['Q6_other'].isna()==False][['Q6','Q6_other']]
q6_idx = df[df['Q6_other'].isna()==False][['Q6','Q6_other']].index
df.iloc[q6_idx][['Q6','Q6_other']]

,Q6,Q6_other
19,Other,Maya
25,Other,"""pentecostal"" spanish survey"
40,Other,"Catholic, Protestant, and Athiest"
47,Other,Pagan
73,Other,Christian
162,Other,Armenian Apostolic
165,Other,baptist
191,Other,Agnostic
229,Other,Christan
238,Other,Agnostic


In [164]:
df['Q6'].unique()

array(['Protestant / Other Christian Denomination', nan, 'Catholic',
       'Atheist', 'Other', 'Buddhism', 'Judaism', 'Islam', 'Hinduism'],
      dtype=object)

Thankfully there aren't too many "Other" answers. A lot of these are Christian, which should be mapped to the "Protestant / Other Christian Denomination" category. We'll do the same for the values of:

 - "pentecostal" spanish survey
 - Catholic, Protestant, and Athiest
 - Armenian Apostolic
 - baptist
 - Eastern Orthodox
 - Adventista
 - Apolistic Pentecostal
 - Russian Orthodox

All other values we'll leave in the "Other" category, and then drop the "Q6_other" column

In [165]:
# Replace values in Q6 based on values in Q6_other
christian_list = ['Christian', '"pentecostal" spanish survey', 'Catholic, Protestant, and Athiest', 'Armenian Apostolic', 'baptist', 'Eastern Orthodox', 'Adventista', 'Apolistic Pentecostal', 'Russian Orthodox']
df.loc[df['Q6_other'].isin(christian_list), 'Q6'] = 'Protestant / Other Christian Denomination'

In [166]:
# Check our work
df.iloc[q6_idx][['Q6','Q6_other']]

,Q6,Q6_other
19,Other,Maya
25,Protestant / Other Christian Denomination,"""pentecostal"" spanish survey"
40,Protestant / Other Christian Denomination,"Catholic, Protestant, and Athiest"
47,Other,Pagan
73,Protestant / Other Christian Denomination,Christian
162,Protestant / Other Christian Denomination,Armenian Apostolic
165,Protestant / Other Christian Denomination,baptist
191,Other,Agnostic
229,Other,Christan
238,Other,Agnostic


In [167]:
# Drop Q6_other
df = df.drop(['index','Q6_other'], axis=1)

In [168]:
df.columns

Index(['UserLanguage', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9',
       'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19',
       'Q20'],
      dtype='object')

Take the moment now to map the question answers to question numbers. This will be useful for our visualizations later.

In [169]:
#list(df.iloc[:,15:-1].columns)
unique_values = []
for column_idx in range(len(df.columns)):
    unique_values.append(np.unique(list(df.iloc[:,column_idx])))
question_answers = dict(zip(df.columns, unique_values))

Q5 and Q17 are "select all that apply" questions, so I'll expect a lot of variance in the replies.

In [170]:
display(df['Q5'].unique())
display(df['Q17'].unique())

array(['Black or African American,American Indian or Alaska Native', nan,
       'Asian', 'Hispanic or Latino/a', 'White',
       'Native Hawaiian or Pacific Islander',
       'White,Black or African American', 'White,Hispanic or Latino/a',
       'American Indian or Alaska Native', 'Black or African American',
       'Black or African American,Hispanic or Latino/a',
       'White,Hispanic or Latino/a,Asian,Native Hawaiian or Pacific Islander',
       'Other',
       'Black or African American,American Indian or Alaska Native,Other',
       'White,American Indian or Alaska Native', 'White,Asian',
       'White,Black or African American,American Indian or Alaska Native',
       'Hispanic or Latino/a,Asian',
       'Hispanic or Latino/a,American Indian or Alaska Native',
       'White,Black or African American,Hispanic or Latino/a',
       'Black or African American,Native Hawaiian or Pacific Islander',
       'White,Native Hawaiian or Pacific Islander', 'Asian,Other'],
      dtype=objec

array([nan,
       "Rhode Island Secretary of State's Website,Social Media,Friends/Family",
       'Friends/Family', 'Social Media',
       "Rhode Island Secretary of State's Website",
       "Rhode Island Secretary of State's Website,Social Media,Religious/Community Organizations",
       'The Voter Information Handbook sent to my house',
       'Social Media,Friends/Family', 'Religious/Community Organizations',
       'The Voter Information Handbook sent to my house,Friends/Family',
       "Rhode Island Secretary of State's Website,The Voter Information Handbook sent to my house,Friends/Family",
       "Rhode Island Secretary of State's Website,The Voter Information Handbook sent to my house,Social Media",
       "Rhode Island Secretary of State's Website,211 Voter Hotline,Friends/Family",
       "Rhode Island Secretary of State's Website,Social Media",
       'The Voter Information Handbook sent to my house,Social Media,Friends/Family',
       '211 Voter Hotline,The Voter Informatio

A lot of the values in these columns are actually multiple values separated by a comma, all in the same cell. We need to figure out how to separate these out into separate columns so we can properly encode them.

In [171]:
df['Q5']\
    .str.split(',', expand=True)\
    .stack(future_stack=True)\
    .str.get_dummies()\
    .groupby(level=0).sum()

,American Indian or Alaska Native,Asian,Black or African American,Hispanic or Latino/a,Native Hawaiian or Pacific Islander,Other,White
0,1,0,1,0,0,0,0
1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
553,0,0,0,0,0,1,0
554,0,0,0,0,0,0,1
555,0,0,0,0,0,0,1
556,0,0,0,1,0,0,0


In [172]:
df['Q17']\
    .str.split(',', expand=True)\
    .stack(future_stack=True)\
    .str.get_dummies()\
    .groupby(level=0).sum()

,211 Voter Hotline,Friends/Family,Religious/Community Organizations,Rhode Island Secretary of State's Website,Social Media,The Voter Information Handbook sent to my house
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,1,0,1,1,0
3,0,1,0,0,0,0
4,0,1,0,1,1,0
...,...,...,...,...,...,...
553,0,0,0,1,0,0
554,0,0,0,1,0,0
555,0,1,0,0,0,0
556,0,1,0,0,1,0


Now to apply it with the rest of our column encodings:

In [173]:
#df['Q2'].str.get_dummies().add_prefix('Q2_')
transformed_data = pd.DataFrame(df['Q1'])

for i in range(2,len(df.columns)): 
    column = 'Q' + str(i)
    if (i != 5 and i != 17):
        sub_df = df[column].str.get_dummies().add_prefix(column+"_")

    else:
        sub_df = df[column]\
        .str.split(',', expand=True)\
        .stack(future_stack=True)\
        .str.get_dummies()\
        .groupby(level=0).sum().add_prefix(column+"_")

    transformed_data = pd.concat([transformed_data, sub_df], axis=1)
display(transformed_data)

,Q1,Q2_18-24,Q2_25-29,Q2_30-39,Q2_40-49,Q2_50-64,Q2_65 or older,Q3_Female,Q3_Male,Q3_Non-binary / Other,...,Q19_Definitely,Q19_Definitely Not,Q19_I would need more information to make a decision,Q19_Probably,Q19_Probably Not,Q20_Definitely,Q20_Definitely Not,Q20_I would need more information to make a decision,Q20_Probably,Q20_Probably Not
0,5,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
1,15,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
2,9,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
3,1,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
4,3,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,5,0,0,0,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
554,5,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
555,5,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
556,5,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0


In [174]:
# also create a mapping dictionary of Q##: [answer choices]
question_choices = {key: [] for key in question_mapping.keys()}

for column_name in transformed_data.columns[1:]:
    pattern_qnumber = re.compile(r"Q\d+") #match on letter Q and then 1 or more digits (0-9)
    mo_1 = pattern_qnumber.search(column_name)
    question_number = mo_1.group()

    pattern_qchoice = re.compile("_.+", re.DOTALL | re.VERBOSE)
    mo_2 = pattern_qchoice.search(column_name)
    #match = re.match(pattern, column_name)
    question_choice = mo_2.group()[1:]
    
    question_choices[question_number].append(question_choice)
question_choices['Q1'] = list(range(1,16))
# export to json so we can use it in our visualizations/dashboarding later
with open('../data/question_choices.json', 'w') as f:
    json.dump(question_choices, f)


To get it ready for GIS work, we need to aggregate all the data by ward. In the end, we want each row to represent a ward, and each column to represent a sum.

In [175]:
grouped_data = transformed_data.groupby(by='Q1', as_index=False).sum()
display(grouped_data.head())

,Q1,Q2_18-24,Q2_25-29,Q2_30-39,Q2_40-49,Q2_50-64,Q2_65 or older,Q3_Female,Q3_Male,Q3_Non-binary / Other,...,Q19_Definitely,Q19_Definitely Not,Q19_I would need more information to make a decision,Q19_Probably,Q19_Probably Not,Q20_Definitely,Q20_Definitely Not,Q20_I would need more information to make a decision,Q20_Probably,Q20_Probably Not
0,1,15,24,15,7,9,10,45,32,3,...,63,2,4,7,2,31,6,14,15,12
1,11,1,3,6,5,6,2,16,7,0,...,16,0,1,1,3,6,1,8,6,0
2,12,1,2,9,4,5,6,11,16,0,...,14,0,6,6,1,13,3,5,5,1
3,14,4,3,9,7,9,4,20,15,0,...,20,2,8,2,4,12,4,8,8,4
4,15,5,4,13,3,2,0,15,11,1,...,15,3,2,4,2,11,3,2,8,2


That is for the raw counts, maybe we also want to do percentages? Therefore each column value needs to be the percentage for that specific ward. We want it to easily answer questions such as "What percentage of Ward 11 residents responded "Definitely" for Q19?"

In [176]:
# Create an emppty dictionary for counts per question per district, to be used in the next code cell
question_mapping.keys()
response_count = {key: 0 for key in question_mapping.keys()}
for key in ['UserLanguage', 'Unnamed: 21']: 
    if key in response_count: del response_count[key]
response_count

{'Q1': 0,
 'Q2': 0,
 'Q3': 0,
 'Q4': 0,
 'Q5': 0,
 'Q6': 0,
 'Q7': 0,
 'Q8': 0,
 'Q9': 0,
 'Q10': 0,
 'Q11': 0,
 'Q12': 0,
 'Q13': 0,
 'Q14': 0,
 'Q15': 0,
 'Q16': 0,
 'Q17': 0,
 'Q18': 0,
 'Q19': 0,
 'Q20': 0}

In [177]:
# going to use regex to match on column name strings
ward_totals = {}
pattern = r"Q\d+" #match on letter Q and then 1 or more digits (0-9)
for index, row in grouped_data.iterrows():
    ward_response_count = response_count.copy() # reset all counts to 0
    for column_idx in range(len(grouped_data.columns)):
        for key in ward_response_count:
            column_name = grouped_data.columns[column_idx]
            match = re.match(pattern, column_name)
            question_number = match.group()
            if key == question_number: # if the key equals the question number we pulled from the column name
                if key == 'Q1': # don't want to add district numbers, need to handle Q10, Q11, etc.
                    ward_response_count[key] = str(row.iloc[column_idx])
                else: 
                    ward_response_count[key] += int(row[column_idx])
            else: pass
    ward_totals[ward_response_count['Q1']] = ward_response_count
for key in ward_totals: print(ward_totals[key])

{'Q1': '1', 'Q2': 80, 'Q3': 80, 'Q4': 80, 'Q5': 82, 'Q6': 52, 'Q7': 78, 'Q8': 77, 'Q9': 79, 'Q10': 80, 'Q11': 80, 'Q12': 79, 'Q13': 79, 'Q14': 79, 'Q15': 79, 'Q16': 79, 'Q17': 119, 'Q18': 79, 'Q19': 78, 'Q20': 78}
{'Q1': '11', 'Q2': 23, 'Q3': 23, 'Q4': 22, 'Q5': 24, 'Q6': 16, 'Q7': 22, 'Q8': 22, 'Q9': 21, 'Q10': 21, 'Q11': 22, 'Q12': 20, 'Q13': 21, 'Q14': 22, 'Q15': 21, 'Q16': 21, 'Q17': 35, 'Q18': 21, 'Q19': 21, 'Q20': 21}
{'Q1': '12', 'Q2': 27, 'Q3': 27, 'Q4': 27, 'Q5': 27, 'Q6': 19, 'Q7': 26, 'Q8': 25, 'Q9': 26, 'Q10': 26, 'Q11': 26, 'Q12': 26, 'Q13': 26, 'Q14': 27, 'Q15': 27, 'Q16': 27, 'Q17': 37, 'Q18': 25, 'Q19': 27, 'Q20': 27}
{'Q1': '14', 'Q2': 36, 'Q3': 36, 'Q4': 36, 'Q5': 36, 'Q6': 25, 'Q7': 36, 'Q8': 34, 'Q9': 34, 'Q10': 34, 'Q11': 34, 'Q12': 34, 'Q13': 35, 'Q14': 35, 'Q15': 36, 'Q16': 36, 'Q17': 51, 'Q18': 36, 'Q19': 36, 'Q20': 36}
{'Q1': '15', 'Q2': 27, 'Q3': 27, 'Q4': 27, 'Q5': 31, 'Q6': 19, 'Q7': 27, 'Q8': 27, 'Q9': 26, 'Q10': 27, 'Q11': 27, 'Q12': 26, 'Q13': 27, 'Q14': 

C:\Users\DWang\AppData\Local\Temp\ipykernel_14236\2945852485.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ward_response_count[key] += int(row[column_idx])


Now we have the raw total counts per question. To get percentages we need to divide each question answer choice column by its question total.

In [178]:
grouped_percentage_data = grouped_data.copy()
grouped_percentage_data = grouped_percentage_data.astype(float)
for index, row in grouped_data.iterrows():
    for column_idx in range(len(grouped_data.columns)):
        for key in ward_totals: # key = district #
            if key == str(row['Q1']): # if the district info aligns
                for question in ward_totals[key]: #iterate through nested dictionary of totals, question --> "Q1", "Q2"...
                    column_name = grouped_data.columns[column_idx]
                    match = re.match(pattern, column_name)
                    column_question_number = match.group()
                    if question == column_question_number and question != "Q1": # if the question aligns
                        # divide the raw count in grouped_percentage_data by the total in ward_totals
                        grouped_percentage_data.at[index, column_name] = row.iloc[column_idx] / ward_totals[key][question]
                    else: pass
            else: pass
grouped_percentage_data['Q1'] = grouped_percentage_data['Q1'].astype(int).astype(str)
display(grouped_percentage_data)

,Q1,Q2_18-24,Q2_25-29,Q2_30-39,Q2_40-49,Q2_50-64,Q2_65 or older,Q3_Female,Q3_Male,Q3_Non-binary / Other,...,Q19_Definitely,Q19_Definitely Not,Q19_I would need more information to make a decision,Q19_Probably,Q19_Probably Not,Q20_Definitely,Q20_Definitely Not,Q20_I would need more information to make a decision,Q20_Probably,Q20_Probably Not
0,1,0.187500,0.300000,0.187500,0.087500,0.112500,0.125000,0.562500,0.400000,0.037500,...,0.807692,0.025641,0.051282,0.089744,0.025641,0.397436,0.076923,0.179487,0.192308,0.153846
1,11,0.043478,0.130435,0.260870,0.217391,0.260870,0.086957,0.695652,0.304348,0.000000,...,0.761905,0.000000,0.047619,0.047619,0.142857,0.285714,0.047619,0.380952,0.285714,0.000000
2,12,0.037037,0.074074,0.333333,0.148148,0.185185,0.222222,0.407407,0.592593,0.000000,...,0.518519,0.000000,0.222222,0.222222,0.037037,0.481481,0.111111,0.185185,0.185185,0.037037
3,14,0.111111,0.083333,0.250000,0.194444,0.250000,0.111111,0.555556,0.416667,0.000000,...,0.555556,0.055556,0.222222,0.055556,0.111111,0.333333,0.111111,0.222222,0.222222,0.111111
4,15,0.185185,0.148148,0.481481,0.111111,0.074074,0.000000,0.555556,0.407407,0.037037,...,0.576923,0.115385,0.076923,0.153846,0.076923,0.423077,0.115385,0.076923,0.307692,0.076923
5,3,0.097222,0.152778,0.291667,0.152778,0.222222,0.083333,0.625000,0.375000,0.000000,...,0.720588,0.044118,0.044118,0.132353,0.058824,0.264706,0.088235,0.191176,0.352941,0.102941
6,5,0.060241,0.048193,0.349398,0.168675,0.216867,0.156627,0.542169,0.421687,0.024096,...,0.582278,0.113924,0.088608,0.139241,0.075949,0.316456,0.050633,0.215190,0.253165,0.164557
7,6,0.101449,0.188406,0.231884,0.159420,0.188406,0.130435,0.681159,0.304348,0.014493,...,0.523077,0.153846,0.092308,0.169231,0.061538,0.257576,0.121212,0.196970,0.257576,0.166667
8,7,0.000000,0.000000,0.000000,0.250000,0.500000,0.250000,0.833333,0.166667,0.000000,...,0.500000,0.000000,0.083333,0.333333,0.083333,0.500000,0.166667,0.250000,0.083333,0.000000
9,8,0.116279,0.093023,0.197674,0.139535,0.290698,0.162791,0.511628,0.488372,0.000000,...,0.505882,0.105882,0.141176,0.164706,0.082353,0.337349,0.084337,0.204819,0.277108,0.096386


To measure oversampling, we want to count the total number of responses total we received for each ward. This will show us if some wards had way more responses than others.

In [179]:
ward_totals = transformed_data\
.groupby("Q1")\
.count()\
.iloc[:,1].rename("Count").copy()
ward_totals = pd.DataFrame(ward_totals)
ward_totals

,Count
Q1,
1,80
11,23
12,27
14,36
15,27
3,72
5,83
6,69
7,12


In [180]:
ward_totals['Percentage'] = ward_totals / sum(ward_totals['Count'])
ward_totals

,Count,Percentage
Q1,,
1,80,0.143627
11,23,0.041293
12,27,0.048474
14,36,0.064632
15,27,0.048474
3,72,0.129264
5,83,0.149013
6,69,0.123878
7,12,0.021544


Now to join it with our GIS data

In [181]:
# Loading in GIS shape files
gdf_pvd = gpd.read_file("../data/BND_PVD_Wards_2022.shp")
gdf_pvd

,ID,DISTRICT,DISTRICTN,Shape_STAr,Shape_STLe,geometry
0,1,1,1.0,4.626871e+07,37132.487547,"POLYGON ((353880.285 269409.163, 354275.176 26..."
1,2,2,2.0,6.857744e+07,45920.952108,"POLYGON ((353740.517 269601.048, 353659.443 26..."
2,3,3,3.0,3.952432e+07,33051.393756,"POLYGON ((352200.695 272587.857, 352185.706 27..."
3,4,4,4.0,4.590033e+07,37093.253292,"POLYGON ((350897.885 273804.377, 350888.071 27..."
4,5,5,5.0,4.342423e+07,37105.452695,"POLYGON ((341271.842 272989.464, 341151.392 27..."
5,6,6,6.0,2.091428e+07,25538.226701,"POLYGON ((341072.364 272964.21, 341151.392 272..."
6,7,7,7.0,2.983633e+07,30271.723284,"POLYGON ((335779.467 269938.082, 335904.304 26..."
7,8,8,8.0,3.567477e+07,39440.578601,"POLYGON ((349492.58 256111.482, 349458.761 256..."
8,9,9,9.0,3.798385e+07,38994.863784,"POLYGON ((352102.808 251083.51, 352075.79 2510..."
9,10,10,10.0,6.951750e+07,46448.983800,"POLYGON ((363622.874 254988.804, 363553.576 25..."


In [182]:
# Merging the dataframes
gis_data = gdf_pvd.merge(grouped_data, left_on='DISTRICT', right_on='Q1')
gis_data_percent = gdf_pvd.merge(grouped_percentage_data, left_on='DISTRICT', right_on='Q1')
gis_data_totals = gdf_pvd.merge(ward_totals, left_on='DISTRICT', right_on='Q1')
display(gis_data)
display(gis_data_percent)
display(gis_data_totals)

,ID,DISTRICT,DISTRICTN,Shape_STAr,Shape_STLe,geometry,Q1,Q2_18-24,Q2_25-29,Q2_30-39,...,Q19_Definitely,Q19_Definitely Not,Q19_I would need more information to make a decision,Q19_Probably,Q19_Probably Not,Q20_Definitely,Q20_Definitely Not,Q20_I would need more information to make a decision,Q20_Probably,Q20_Probably Not
0,1,1,1.0,4.626871e+07,37132.487547,"POLYGON ((353880.285 269409.163, 354275.176 26...",1,15,24,15,...,63,2,4,7,2,31,6,14,15,12
1,3,3,3.0,3.952432e+07,33051.393756,"POLYGON ((352200.695 272587.857, 352185.706 27...",3,7,11,21,...,49,3,3,9,4,18,6,13,24,7
2,5,5,5.0,4.342423e+07,37105.452695,"POLYGON ((341271.842 272989.464, 341151.392 27...",5,5,4,29,...,46,9,7,11,6,25,4,17,20,13
3,6,6,6.0,2.091428e+07,25538.226701,"POLYGON ((341072.364 272964.21, 341151.392 272...",6,7,13,16,...,34,10,6,11,4,17,8,13,17,11
4,7,7,7.0,2.983633e+07,30271.723284,"POLYGON ((335779.467 269938.082, 335904.304 26...",7,0,0,0,...,6,0,1,4,1,6,2,3,1,0
5,8,8,8.0,3.567477e+07,39440.578601,"POLYGON ((349492.58 256111.482, 349458.761 256...",8,10,8,17,...,43,9,12,14,7,28,7,17,23,8
6,9,9,9.0,3.798385e+07,38994.863784,"POLYGON ((352102.808 251083.51, 352075.79 2510...",9,9,1,6,...,29,2,3,6,1,24,0,5,9,3
7,11,11,11.0,3.199302e+07,27702.548389,"POLYGON ((351227.874 261975.2, 351115.2 262319...",11,1,3,6,...,16,0,1,1,3,6,1,8,6,0
8,12,12,12.0,2.799678e+07,32525.697208,"POLYGON ((344897.215 270074.402, 344812.718 27...",12,1,2,9,...,14,0,6,6,1,13,3,5,5,1
9,14,14,14.0,2.632960e+07,28610.798359,"POLYGON ((346817.996 274470.889, 346591.433 27...",14,4,3,9,...,20,2,8,2,4,12,4,8,8,4


,ID,DISTRICT,DISTRICTN,Shape_STAr,Shape_STLe,geometry,Q1,Q2_18-24,Q2_25-29,Q2_30-39,...,Q19_Definitely,Q19_Definitely Not,Q19_I would need more information to make a decision,Q19_Probably,Q19_Probably Not,Q20_Definitely,Q20_Definitely Not,Q20_I would need more information to make a decision,Q20_Probably,Q20_Probably Not
0,1,1,1.0,4.626871e+07,37132.487547,"POLYGON ((353880.285 269409.163, 354275.176 26...",1,0.187500,0.300000,0.187500,...,0.807692,0.025641,0.051282,0.089744,0.025641,0.397436,0.076923,0.179487,0.192308,0.153846
1,3,3,3.0,3.952432e+07,33051.393756,"POLYGON ((352200.695 272587.857, 352185.706 27...",3,0.097222,0.152778,0.291667,...,0.720588,0.044118,0.044118,0.132353,0.058824,0.264706,0.088235,0.191176,0.352941,0.102941
2,5,5,5.0,4.342423e+07,37105.452695,"POLYGON ((341271.842 272989.464, 341151.392 27...",5,0.060241,0.048193,0.349398,...,0.582278,0.113924,0.088608,0.139241,0.075949,0.316456,0.050633,0.215190,0.253165,0.164557
3,6,6,6.0,2.091428e+07,25538.226701,"POLYGON ((341072.364 272964.21, 341151.392 272...",6,0.101449,0.188406,0.231884,...,0.523077,0.153846,0.092308,0.169231,0.061538,0.257576,0.121212,0.196970,0.257576,0.166667
4,7,7,7.0,2.983633e+07,30271.723284,"POLYGON ((335779.467 269938.082, 335904.304 26...",7,0.000000,0.000000,0.000000,...,0.500000,0.000000,0.083333,0.333333,0.083333,0.500000,0.166667,0.250000,0.083333,0.000000
5,8,8,8.0,3.567477e+07,39440.578601,"POLYGON ((349492.58 256111.482, 349458.761 256...",8,0.116279,0.093023,0.197674,...,0.505882,0.105882,0.141176,0.164706,0.082353,0.337349,0.084337,0.204819,0.277108,0.096386
6,9,9,9.0,3.798385e+07,38994.863784,"POLYGON ((352102.808 251083.51, 352075.79 2510...",9,0.214286,0.023810,0.142857,...,0.707317,0.048780,0.073171,0.146341,0.024390,0.585366,0.000000,0.121951,0.219512,0.073171
7,11,11,11.0,3.199302e+07,27702.548389,"POLYGON ((351227.874 261975.2, 351115.2 262319...",11,0.043478,0.130435,0.260870,...,0.761905,0.000000,0.047619,0.047619,0.142857,0.285714,0.047619,0.380952,0.285714,0.000000
8,12,12,12.0,2.799678e+07,32525.697208,"POLYGON ((344897.215 270074.402, 344812.718 27...",12,0.037037,0.074074,0.333333,...,0.518519,0.000000,0.222222,0.222222,0.037037,0.481481,0.111111,0.185185,0.185185,0.037037
9,14,14,14.0,2.632960e+07,28610.798359,"POLYGON ((346817.996 274470.889, 346591.433 27...",14,0.111111,0.083333,0.250000,...,0.555556,0.055556,0.222222,0.055556,0.111111,0.333333,0.111111,0.222222,0.222222,0.111111


,ID,DISTRICT,DISTRICTN,Shape_STAr,Shape_STLe,geometry,Count,Percentage
0,1,1,1.0,4.626871e+07,37132.487547,"POLYGON ((353880.285 269409.163, 354275.176 26...",80,0.143627
1,3,3,3.0,3.952432e+07,33051.393756,"POLYGON ((352200.695 272587.857, 352185.706 27...",72,0.129264
2,5,5,5.0,4.342423e+07,37105.452695,"POLYGON ((341271.842 272989.464, 341151.392 27...",83,0.149013
3,6,6,6.0,2.091428e+07,25538.226701,"POLYGON ((341072.364 272964.21, 341151.392 272...",69,0.123878
4,7,7,7.0,2.983633e+07,30271.723284,"POLYGON ((335779.467 269938.082, 335904.304 26...",12,0.021544
5,8,8,8.0,3.567477e+07,39440.578601,"POLYGON ((349492.58 256111.482, 349458.761 256...",86,0.154399
6,9,9,9.0,3.798385e+07,38994.863784,"POLYGON ((352102.808 251083.51, 352075.79 2510...",42,0.075404
7,11,11,11.0,3.199302e+07,27702.548389,"POLYGON ((351227.874 261975.2, 351115.2 262319...",23,0.041293
8,12,12,12.0,2.799678e+07,32525.697208,"POLYGON ((344897.215 270074.402, 344812.718 27...",27,0.048474
9,14,14,14.0,2.632960e+07,28610.798359,"POLYGON ((346817.996 274470.889, 346591.433 27...",36,0.064632


Export the data sets

In [183]:
transformed_data.to_csv('../data/transformed_data.csv', index=False)
grouped_data.to_csv('../data/grouped_data.csv', index=False)
gis_data.to_csv('../data/gis_data.csv', index=False)
gis_data_percent.to_csv('../data/gis_data_percent.csv', index=False)
gis_data_totals.to_csv('../data/gis_data_totals.csv', index=False)